In [1]:
import requests

# Criar uma lista de anos para os quais você deseja baixar os arquivos
anos = list(range(2012, 2024))

for ano in anos:
    # Construir o URL dinamicamente com base no ano
    url = f'https://www.bcb.gov.br/pda/desig/planilha_{ano}.zip'
    
    # Baixar o arquivo ZIP
    r = requests.get(url, allow_redirects=True)

    # Especificar o nome do arquivo ZIP para salvar
    arquivo_zip = f'planilha_{ano}.zip'

    # Salvar o arquivo ZIP
    with open(arquivo_zip, 'wb') as f:
        f.write(r.content)
    
    print(f"Arquivo {arquivo_zip} baixado com sucesso!")

Arquivo planilha_2012.zip baixado com sucesso!
Arquivo planilha_2013.zip baixado com sucesso!
Arquivo planilha_2014.zip baixado com sucesso!
Arquivo planilha_2015.zip baixado com sucesso!
Arquivo planilha_2016.zip baixado com sucesso!
Arquivo planilha_2017.zip baixado com sucesso!
Arquivo planilha_2018.zip baixado com sucesso!
Arquivo planilha_2019.zip baixado com sucesso!
Arquivo planilha_2020.zip baixado com sucesso!
Arquivo planilha_2021.zip baixado com sucesso!
Arquivo planilha_2022.zip baixado com sucesso!
Arquivo planilha_2023.zip baixado com sucesso!


In [2]:
import zipfile
import os

In [3]:
def extrair_zip_por_ano(ano):
    zip_file_name = f'planilha_{ano}.zip'
    extracao_dir = f'planilha_{ano}'

    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(extracao_dir)
        
    extracted_files = os.listdir(extracao_dir)
    for file_name in extracted_files:
        print(f"Arquivo extraído: {file_name}")

In [ ]:
extrair_zip_por_ano(2013) #trocar o ano pelo ano que você deseja extrair o arquivo
#Se você quiser extrair para mais de um ano de uma vez, use a linha abaixo

In [6]:
anos = list(range(2012, 2024))

for ano in anos:
    extrair_zip_por_ano(ano)

Arquivo extraído: planilha_201206.csv
Arquivo extraído: planilha_201212.csv
Arquivo extraído: planilha_201207.csv
Arquivo extraído: planilha_201211.csv
Arquivo extraído: planilha_201210.csv
Arquivo extraído: planilha_201209.csv
Arquivo extraído: planilha_201208.csv
Arquivo extraído: planilha_201311.csv
Arquivo extraído: planilha_201305.csv
Arquivo extraído: planilha_201304.csv
Arquivo extraído: planilha_201310.csv
Arquivo extraído: planilha_201306.csv
Arquivo extraído: planilha_201312.csv
Arquivo extraído: planilha_201307.csv
Arquivo extraído: planilha_201303.csv
Arquivo extraído: planilha_201302.csv
Arquivo extraído: planilha_201301.csv
Arquivo extraído: planilha_201309.csv
Arquivo extraído: planilha_201308.csv
Arquivo extraído: planilha_201401.csv
Arquivo extraído: planilha_201403.csv
Arquivo extraído: planilha_201402.csv
Arquivo extraído: planilha_201406.csv
Arquivo extraído: planilha_201412.csv
Arquivo extraído: planilha_201407.csv
Arquivo extraído: planilha_201411.csv
Arquivo extr

In [7]:
import pandas as pd

In [8]:
def concatenar_csvs(diretorio):
    frames = []

    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(diretorio, arquivo)
            
            df = pd.read_csv(caminho_arquivo, encoding='utf-8', delimiter = ";")
            
            frames.append(df)

    df_concatenado = pd.concat(frames, ignore_index=True)

    return df_concatenado

**A partir daqui, a concatenação é feita planilha por planilha para não sobrecarregar

***Sugiro usar o to_csv apenas após fazer as transformações necessárias

****Mas, se você quiser automatizar e fazer tudo de uma vez, tem um código para isso após a linha dicionario_2012.head(5)

In [9]:
dados_2012 = concatenar_csvs("planilha_2012")

In [11]:
dados_2012['a_vencer_de_361_ate_1080_dias'] = dados_2012['a_vencer_de_361_ate_1080_dias'].str.replace(',', '.')

In [12]:
dados_2012['a_vencer_de_361_ate_1080_dias'] = pd.to_numeric(dados_2012['a_vencer_de_361_ate_1080_dias'], errors='coerce')

In [13]:
dados_2012['a_vencer_de_361_ate_1080_dias']

0             1203.94
1             2911.48
2                0.00
3          6558429.77
4          1507329.67
              ...    
3230178          0.00
3230179      11954.60
3230180          0.00
3230181     100518.44
3230182          0.00
Name: a_vencer_de_361_ate_1080_dias, Length: 3230183, dtype: float64

In [18]:
dados_2012['data_base'] = pd.to_datetime(dados_2012['data_base'], format = "%Y-%m-%d")

In [20]:
dados_2012['data_base'] = dados_2012['data_base'].dt.to_period('M')

In [21]:
dados_2012['data_base']

0          2012-06
1          2012-06
2          2012-06
3          2012-06
4          2012-06
            ...   
3230178    2012-08
3230179    2012-08
3230180    2012-08
3230181    2012-08
3230182    2012-08
Name: data_base, Length: 3230183, dtype: period[M]

In [14]:
meu_dicionario = dados_2012.groupby(['data_base'])['a_vencer_de_361_ate_1080_dias'].sum().reset_index()

In [15]:
dicionario_2012 = pd.DataFrame(meu_dicionario)

In [16]:
dicionario_2012.head(5)

,data_base,a_vencer_de_361_ate_1080_dias
0,2012-06-30,5.819965e+11
1,2012-07-31,5.952558e+11
2,2012-08-31,6.032827e+11
3,2012-09-30,6.024891e+11
4,2012-10-31,6.318362e+11


In [27]:
def automatizacao_groupby(ano):
    dados = concatenar_csvs(f"planilha_{ano}")
    
    dados['a_vencer_de_361_ate_1080_dias'] = dados['a_vencer_de_361_ate_1080_dias'].str.replace(',', '.')
    dados['a_vencer_de_361_ate_1080_dias'] = pd.to_numeric(dados['a_vencer_de_361_ate_1080_dias'], errors='coerce')
    
    dados['data_base'] = pd.to_datetime(dados['data_base'], format="%Y-%m-%d")
    dados['data_base'] = dados['data_base'].dt.to_period('M')
    
    meu_dicionario = dados.groupby(['data_base'])['a_vencer_de_361_ate_1080_dias'].sum().reset_index()
    dicionario = pd.DataFrame(meu_dicionario)
    
    dataframes.append(dicionario)
    
dataframes = []

In [28]:
for ano in range(2012, 2024):
    automatizacao_groupby(ano)

In [29]:
dataframes

[  data_base  a_vencer_de_361_ate_1080_dias
 0   2012-06                   5.819965e+11
 1   2012-07                   5.952558e+11
 2   2012-08                   6.032827e+11
 3   2012-09                   6.024891e+11
 4   2012-10                   6.318362e+11
 5   2012-11                   6.384308e+11
 6   2012-12                   6.508781e+11,
    data_base  a_vencer_de_361_ate_1080_dias
 0    2013-01                   6.539443e+11
 1    2013-02                   6.579861e+11
 2    2013-03                   6.481237e+11
 3    2013-04                   6.779428e+11
 4    2013-05                   6.898174e+11
 5    2013-06                   6.772245e+11
 6    2013-07                   7.109691e+11
 7    2013-08                   7.136383e+11
 8    2013-09                   7.108068e+11
 9    2013-10                   7.045516e+11
 10   2013-11                   6.838371e+11
 11   2013-12                   7.207290e+11,
    data_base  a_vencer_de_361_ate_1080_dias
 0    2014-01   

In [32]:
resultado_final = pd.concat(dataframes, ignore_index=True)

In [33]:
df_para_estudo = pd.DataFrame(resultado_final)

In [34]:
df_para_estudo.head(5)

,data_base,a_vencer_de_361_ate_1080_dias
0,2012-06,5.819965e+11
1,2012-07,5.952558e+11
2,2012-08,6.032827e+11
3,2012-09,6.024891e+11
4,2012-10,6.318362e+11
